# Workflow as Agent with Reflection and Retry Pattern

**Copyright (c) Microsoft. All rights reserved.**

## Purpose

This notebook demonstrates how to wrap a workflow as an agent using `WorkflowAgent`. It uses a reflection pattern where:

- A **Worker** executor generates responses
- A **Reviewer** executor evaluates them
- If the response is not approved, the Worker regenerates based on feedback
- Only approved responses are emitted to the external consumer
- The workflow completes when idle

## Key Concepts Demonstrated

1. **WorkflowAgent**: Wraps a workflow to behave like a regular agent
2. **Cyclic workflow design**: Worker ↔ Reviewer for iterative improvement
3. **AgentRunUpdateEvent**: Mechanism for emitting approved responses externally
4. **Structured output parsing**: Review feedback using Pydantic
5. **State management**: Pending requests and retry logic

## Prerequisites

- OpenAI account configured and accessible for `OpenAIChatClient`
- Familiarity with `WorkflowBuilder`, `Executor`, `WorkflowContext`, and event handling
- Understanding of how agent messages are generated, reviewed, and re-submitted

## Setup and Imports

Import required libraries for workflow orchestration, agent framework, and structured data handling.

In [1]:
from dataclasses import dataclass
from uuid import uuid4

from agent_framework import (
    AgentRunResponseUpdate,
    AgentRunUpdateEvent,
    ChatClientProtocol,
    ChatMessage,
    Contents,
    Executor,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    handler,
)
from agent_framework.openai import OpenAIChatClient
from pydantic import BaseModel

## Define Data Structures

Create structured request and response classes for communication between Worker and Reviewer.

In [2]:
@dataclass
class ReviewRequest:
    """Structured request passed from Worker to Reviewer for evaluation."""

    request_id: str
    user_messages: list[ChatMessage]
    agent_messages: list[ChatMessage]


@dataclass
class ReviewResponse:
    """Structured response from Reviewer back to Worker."""

    request_id: str
    feedback: str
    approved: bool

## Implement Reviewer Executor

The Reviewer evaluates agent responses against quality criteria:
- **Relevance**: Response addresses the query
- **Accuracy**: Information is correct
- **Clarity**: Response is easy to understand
- **Completeness**: Response covers all aspects

In [3]:
class Reviewer(Executor):
    """Executor that reviews agent responses and provides structured feedback."""

    def __init__(self, id: str, chat_client: ChatClientProtocol) -> None:
        super().__init__(id=id)
        self._chat_client = chat_client

    @handler
    async def review(self, request: ReviewRequest, ctx: WorkflowContext[ReviewResponse]) -> None:
        print(f"Reviewer: Evaluating response for request {request.request_id[:8]}...")

        # Define structured schema for the LLM to return.
        class _Response(BaseModel):
            feedback: str
            approved: bool

        # Construct review instructions and context.
        messages = [
            ChatMessage(
                role=Role.SYSTEM,
                text=(
                    "You are a reviewer for an AI agent. Provide feedback on the "
                    "exchange between a user and the agent. Indicate approval only if:\n"
                    "- Relevance: response addresses the query\n"
                    "- Accuracy: information is correct\n"
                    "- Clarity: response is easy to understand\n"
                    "- Completeness: response covers all aspects\n"
                    "Do not approve until all criteria are satisfied."
                ),
            )
        ]
        # Add conversation history.
        messages.extend(request.user_messages)
        messages.extend(request.agent_messages)

        # Add explicit review instruction.
        messages.append(ChatMessage(role=Role.USER, text="Please review the agent's responses."))

        print("Reviewer: Sending review request to LLM...")
        response = await self._chat_client.get_response(
            messages=messages, response_format=_Response
        )

        parsed = _Response.model_validate_json(response.messages[-1].text)

        print(f"Reviewer: Review complete - Approved: {parsed.approved}")
        print(f"Reviewer: Feedback: {parsed.feedback}")

        # Send structured review result to Worker.
        await ctx.send_message(
            ReviewResponse(
                request_id=request.request_id, feedback=parsed.feedback, approved=parsed.approved
            )
        )

## Implement Worker Executor

The Worker generates responses and incorporates feedback when necessary. It maintains state for pending requests to handle the retry cycle.

In [4]:
class Worker(Executor):
    """Executor that generates responses and incorporates feedback when necessary."""

    def __init__(self, id: str, chat_client: ChatClientProtocol) -> None:
        super().__init__(id=id)
        self._chat_client = chat_client
        self._pending_requests: dict[str, tuple[ReviewRequest, list[ChatMessage]]] = {}

    @handler
    async def handle_user_messages(
        self, user_messages: list[ChatMessage], ctx: WorkflowContext[ReviewRequest]
    ) -> None:
        print("Worker: Received user messages, generating response...")

        # Initialize chat with system prompt.
        messages = [ChatMessage(role=Role.SYSTEM, text="You are a helpful assistant.")]
        messages.extend(user_messages)

        print("Worker: Calling LLM to generate response...")
        response = await self._chat_client.get_response(messages=messages)
        print(f"Worker: Response generated: {response.messages[-1].text}")

        # Add agent messages to context.
        messages.extend(response.messages)

        # Create review request and send to Reviewer.
        request = ReviewRequest(
            request_id=str(uuid4()), user_messages=user_messages, agent_messages=response.messages
        )
        print(f"Worker: Sending response for review (ID: {request.request_id[:8]})")
        await ctx.send_message(request)

        # Track request for possible retry.
        self._pending_requests[request.request_id] = (request, messages)

    @handler
    async def handle_review_response(
        self, review: ReviewResponse, ctx: WorkflowContext[ReviewRequest]
    ) -> None:
        print(
            print(
                f"Worker: Received review for request {review.request_id[:8]} - "
                f"Approved: {review.approved}"
            )
        )

        if review.request_id not in self._pending_requests:
            raise ValueError(f"Unknown request ID in review: {review.request_id}")

        request, messages = self._pending_requests.pop(review.request_id)

        if review.approved:
            print("Worker: Response approved. Emitting to external consumer...")
            contents: list[Contents] = []
            for message in request.agent_messages:
                contents.extend(message.contents)

            # Emit approved result to external consumer via AgentRunUpdateEvent.
            await ctx.add_event(
                AgentRunUpdateEvent(
                    self.id, data=AgentRunResponseUpdate(contents=contents, role=Role.ASSISTANT)
                )
            )
            return

        print(f"Worker: Response not approved. Feedback: {review.feedback}")
        print("Worker: Regenerating response with feedback...")

        # Incorporate review feedback.
        messages.append(ChatMessage(role=Role.SYSTEM, text=review.feedback))
        messages.append(
            ChatMessage(
                role=Role.SYSTEM,
                text="Please incorporate the feedback and regenerate the response.",
            )
        )
        messages.extend(request.user_messages)

        # Retry with updated prompt.
        response = await self._chat_client.get_response(messages=messages)
        print(f"Worker: New response generated: {response.messages[-1].text}")

        messages.extend(response.messages)

        # Send updated request for re-review.
        new_request = ReviewRequest(
            request_id=review.request_id,
            user_messages=request.user_messages,
            agent_messages=response.messages,
        )
        await ctx.send_message(new_request)

        # Track new request for further evaluation.
        self._pending_requests[new_request.request_id] = (new_request, messages)

## Build and Run the Workflow Agent

Create the cyclic workflow (Worker ↔ Reviewer) and wrap it as an agent. The workflow will iteratively improve responses until the Reviewer approves.

In [6]:
async def run_workflow_agent():
    print("Starting Workflow Agent Demo")
    print("=" * 50)

    # Initialize chat clients and executors.
    print("Creating chat client and executors...")
    mini_chat_client = OpenAIChatClient(model_id="gpt-4.1-nano")
    chat_client = OpenAIChatClient(model_id="gpt-4.1")
    reviewer = Reviewer(id="reviewer", chat_client=chat_client)
    worker = Worker(id="worker", chat_client=mini_chat_client)

    print("Building workflow with Worker ↔ Reviewer cycle...")
    agent = (
        WorkflowBuilder()
        .add_edge(worker, reviewer)  # Worker sends responses to Reviewer
        .add_edge(reviewer, worker)  # Reviewer provides feedback to Worker
        .set_start_executor(worker)
        .build()
        .as_agent()  # Wrap workflow as an agent
    )

    print("Running workflow agent with user query...")
    query = (
        "Write code for parallel reading 1 million files on disk and write to a sorted output file."
    )
    print(f"Query: '{query}'")
    print("-" * 50)

    # Run agent in streaming mode to observe incremental updates.
    async for event in agent.run_stream(query):
        print(f"Agent Response: {event}")

    print("=" * 50)
    print("Workflow completed!")


# Run the workflow agent
await run_workflow_agent()

Starting Workflow Agent Demo
Creating chat client and executors...
Building workflow with Worker ↔ Reviewer cycle...
Running workflow agent with user query...
Query: 'Write code for parallel reading 1 million files on disk and write to a sorted output file.'
--------------------------------------------------
Worker: Received user messages, generating response...
Worker: Calling LLM to generate response...
Worker: Response generated: Certainly! Reading 1 million files in parallel and writing their contents to a sorted output file requires efficient handling of concurrency, I/O, and sorting. Here's a Python example using `concurrent.futures` for parallelism and `heapq.merge` for efficient merging of sorted data:

### Assumptions:
- Each file contains sorted data lines.
- You want to produce a single sorted output file containing all lines.
- Files are large but can be processed line-by-line.
- You have sufficient memory and resources for parallel processing.

### Approach:
1. Read all fi

## Try Different Queries

Experiment with different types of queries to see how the reflection pattern works.

In [ ]:
async def test_query(query: str):
    """Test the workflow agent with a specific query."""
    print(f"\nTesting Query: {query}")
    print("=" * 60)

    mini_chat_client = OpenAIChatClient(model_id="gpt-4.1-nano")
    chat_client = OpenAIChatClient(model_id="gpt-4.1")
    reviewer = Reviewer(id="reviewer", chat_client=chat_client)
    worker = Worker(id="worker", chat_client=mini_chat_client)

    agent = (
        WorkflowBuilder()
        .add_edge(worker, reviewer)
        .add_edge(reviewer, worker)
        .set_start_executor(worker)
        .build()
        .as_agent()
    )

    async for event in agent.run_stream(query):
        print(f"Response: {event}")

    print("=" * 60)


# Test with different query types
await test_query("Explain quantum computing in simple terms.")

In [ ]:
# Try a coding question
await test_query("Write a Python function to find the longest palindrome substring.")

In [ ]:
# Try a complex analysis question
await test_query("Compare and contrast microservices and monolithic architectures.")

## Workflow Architecture Visualization

```
┌─────────────────────────────────────────────────────────────┐
│                    Workflow as Agent                        │
│                                                             │
│  ┌─────────┐         ┌──────────┐         ┌──────────┐   │
│  │  User   │────────▶│  Worker  │────────▶│ Reviewer │   │
│  │  Query  │         │          │         │          │   │
│  └─────────┘         └──────────┘         └──────────┘   │
│                            ▲                     │         │
│                            │   Not Approved      │         │
│                            │   (with feedback)   │         │
│                            └─────────────────────┘         │
│                                                             │
│                      Approved ──────────▶ External         │
│                                          Consumer          │
└─────────────────────────────────────────────────────────────┘
```

### Flow:
1. User query enters the workflow
2. Worker generates initial response
3. Reviewer evaluates against quality criteria
4. If not approved: feedback sent to Worker → regenerate
5. If approved: response emitted to external consumer
6. Workflow completes when idle

## Advanced: Custom Review Criteria

Modify the Reviewer's criteria to focus on specific aspects.

In [ ]:
class StrictCodeReviewer(Executor):
    """Reviewer with strict criteria for code-related responses."""

    def __init__(self, id: str, chat_client: ChatClientProtocol) -> None:
        super().__init__(id=id)
        self._chat_client = chat_client

    @handler
    async def review(self, request: ReviewRequest, ctx: WorkflowContext[ReviewResponse]) -> None:
        print(f"StrictCodeReviewer: Evaluating response for request {request.request_id[:8]}...")

        class _Response(BaseModel):
            feedback: str
            approved: bool

        messages = [
            ChatMessage(
                role=Role.SYSTEM,
                text=(
                    "You are a strict code reviewer. Approve only if:\n"
                    "- Code is syntactically correct\n"
                    "- Includes error handling\n"
                    "- Has proper type hints\n"
                    "- Includes docstrings\n"
                    "- Follows best practices (e.g., async/await for I/O)\n"
                    "- Includes usage example"
                ),
            )
        ]
        messages.extend(request.user_messages)
        messages.extend(request.agent_messages)
        messages.append(ChatMessage(role=Role.USER, text="Please review the code response."))

        response = await self._chat_client.get_response(
            messages=messages, response_format=_Response
        )
        parsed = _Response.model_validate_json(response.messages[-1].text)

        print(f"StrictCodeReviewer: Approved: {parsed.approved}")
        print(f"StrictCodeReviewer: Feedback: {parsed.feedback}")

        await ctx.send_message(
            ReviewResponse(
                request_id=request.request_id, feedback=parsed.feedback, approved=parsed.approved
            )
        )


# Test with strict code reviewer
async def test_strict_code_review():
    mini_chat_client = OpenAIChatClient(model_id="gpt-4.1-nano")
    chat_client = OpenAIChatClient(model_id="gpt-4.1")
    reviewer = StrictCodeReviewer(id="strict_reviewer", chat_client=chat_client)
    worker = Worker(id="worker", chat_client=mini_chat_client)

    agent = (
        WorkflowBuilder()
        .add_edge(worker, reviewer)
        .add_edge(reviewer, worker)
        .set_start_executor(worker)
        .build()
        .as_agent()
    )

    async for event in agent.run_stream("Write a function to read a JSON file asynchronously."):
        print(f"Response: {event}")


await test_strict_code_review()

## Summary

This notebook demonstrated:

1. **Workflow as Agent Pattern**: Using `WorkflowBuilder.build().as_agent()` to expose workflows as agents
2. **Cyclic Workflows**: Worker ↔ Reviewer pattern for iterative refinement
3. **Structured Communication**: Using dataclasses and Pydantic for type-safe messaging
4. **State Management**: Tracking pending requests for retry logic
5. **Event Emission**: Using `AgentRunUpdateEvent` to emit approved responses
6. **Quality Control**: Multi-criteria evaluation before accepting responses

### Key Takeaways

- Workflows can behave like agents using `.as_agent()`
- Cyclic workflows enable self-improvement patterns
- Structured outputs ensure reliable inter-executor communication
- Review criteria can be customized per use case
- External consumers only see approved, high-quality responses

### Next Steps

- Add multiple reviewers for different aspects (code quality, security, performance)
- Implement retry limits to prevent infinite loops
- Add metrics tracking (review iterations, approval rate)
- Integrate with AgenticFleet for multi-agent orchestration
- Add human-in-the-loop approval for sensitive operations